In [1]:
import os
import ltspice
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd

In [2]:
# Main Code
from modules.netlist import Netlist
from modules.common import *
from config import *

config['simulation']['savedir'] = create_save_directory(
    config['simulation']['save'], 
    folder_name
)

# Create Netlist object
netlist = Netlist()

# Import data and create inputs
x_train, y_train, x_test, y_test = import_data(config)
create_inputs(x_train, x_test, config)

# Calculate simulation time and initialize weights
config["simulation"]["time"] = calculate_time(len(x_train), len(x_test), config)
config["simulation"]["weights"] = initialize_weights(config)

# Create netlist and save it
netlist.mk_circuit(config)
netlist.save_net(config)

# Print information
print_information(config)

# Run LTspice
run_ltspice(config)

# Import results
data = import_results(config)

# Print shape results
print(f"supposed tlen: {len(y_train) + len(y_test)}")
print(f"data shape: {np.shape(data[0])}")

Netlist saved in C:\Users\ricar\Google Drive\INESC MN\Software\PCSLTS\results\2024-04-16_15.06.41\circuit.cir
 ____    ____  ____   _      _____  ____  
|  _ \  / ___|/ ___| | |    |_   _|/ ___| 
| |_) || |    \___ \ | |      | |  \___ \ 
|  __/ | |___  ___) || |___   | |   ___) |
|_|     \____||____/ |_____|  |_|  |____/ 
                                          
 Python Crossbar Simulation with LTSpice
 
 ---------- 
 
 Author: Ricardo E. Silva
 Research Group: INESC MN
 Licence: MIT
 Version: 0.1
 
 ---------- 
 


 Configuration: 


---- simulation ----
save: C:\Users\ricar\Google Drive\INESC MN\Software\PCSLTS\results
dataset: wqpt
geometry: [[11, 14], [14, 10]]
test_size: 0.2
timestep: 1e-09
freq: 1000000000.0
precision: 10
vin_scale: 1
vout_scale: 1000.0
epochs: 50
learning_rate: 0.2
savedir: C:\Users\ricar\Google Drive\INESC MN\Software\PCSLTS\results\2024-04-16_15.06.41
time: 6.497000000000001e-06

---- opamp ----
power: 1
noninverting: 0

---- resistor ----
A: 1000.0
B: 100.

In [4]:
def update_weights(output, true_output, weights, learning_rate):

    def sigmoid_derivative(x): return x * (1 - x)
   
    layer_errors = [true_output - output[-1]]
    layer_deltas = [layer_errors[-1] * sigmoid_derivative(output[-1])]

    for i in range(len(weights) - 1, 0, -1):
        error = layer_deltas[0].dot(weights[i].T)
        delta = error * sigmoid_derivative(output[i])
        layer_errors.insert(0, error)
        layer_deltas.insert(0, delta)


    # Update weights
    for i in range(len(weights)):
        print(f'output[{i}]: {np.shape(output[i])}')
        print(f'layer_deltas[{i}]: {np.shape(layer_deltas[i])}')
        print(f'weights[{i}]: {np.shape(weights[i])}')
        
        weights[i] += output[i].T.dot(layer_deltas[i]) * learning_rate

    return weights

new_weights = update_weights(
    data, 
    np.concatenate((y_test, y_train), axis=0), 
    config["simulation"]["weights"], 
    0.01
)

ValueError: operands could not be broadcast together with shapes (6497,14) (6497,10) 

In [5]:
geometry = config["simulation"]["geometry"]
weights = [np.ones((rows, cols)) for rows, cols in geometry]

def sigmoid_derivative(x): return x * (1 - x)

layer_errors = [np.concatenate((y_test, y_train), axis=0) - data[-1]]
layer_deltas = [layer_errors[-1] * sigmoid_derivative(data[-1])]

print(np.shape(layer_errors[-1]))
print(np.shape(layer_deltas[-1]))

error = np.dot(layer_deltas[-1], weights[-1].T)
print(np.shape(error))

(6497, 10)
(6497, 10)
(6497, 11)


In [6]:
print(np.shape(weights[-1]))

(11, 10)
